In [1]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

In [17]:
# make page
paper_size = '11x14 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [34]:
db = drawbox.buffer(-15, cap_style=2, join_style=2)

In [35]:
layer = gp.connected_hatchbox(db, angle=45, spacing=0.3, dist_thresh=0.6)

In [36]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.1mm')
sk.geometry(layer)
tolerance=0.1
sk.vpype(f'linemerge --tolerance 0.5mm linesort')
sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-07T20:19:59.381424 
 
 
 
 
 
 <polyline points="133.6675,1211.7165 132.2835,1210.3325 132.2835,1208.729 135.271,1211.7165 136.8745,1211.7165 132.2835,1207.1255 132.2835,1205.522 138.478,1211.7165 140.0815,1211.7165 132.2835,1203.9185 132.2835,1202.315 141.685,1211.7165 143.2886,1211.7165 132.2835,1200.7114 132.2835,1199.1079 144.8921,1211.7165 146.4956,1211.7165 132.2835,1197.5044 132.2835,1195.9009 148.0991,1211.7165 149.7026,1211.7165 132.2835,1194.2974 132.2835,1192.6939 151.3061,1211.7165 152.9097,1211.7165 132.2835,1191.0903 132.2835,1189.4868 154.5132,1211.7165 156.1167,1211.7165 132.2835,1187.8833 132.2835,1186.2798 157.7202,1211.7165 159.3237,1211.7165 132.2835,1184.6763 132.2835,1183.0727 160.9273,1211.7165 162.5308,1211.7165 132.2835,1181.4692 132.2835,1179.8657 164.1343,1211.7165 165.7378,1211.7165 132.2835,1178.2622 132.2835,1176.6587 167.3413,1211.7165 168.9448,1211.7165 132.2835,1175.0552 132.2835,1173.4516 170.5484,1211.7165 172.1519,1211.7165 132.2835,1171.8481 132.2835,1170.2446 173.7554,1211.7165 175.3589,1211.7165 132.2835,1168.6411 132.2835,1167.0376 176.9624,1211.7165 178.5659,1211.7165 132.2835,1165.4341 132.2835,1163.8305 180.1695,1211.7165 181.773,1211.7165 132.2835,1162.227 132.2835,1160.6235 183.3765,1211.7165 184.98,1211.7165 132.2835,1159.02 132.2835,1157.4165 186.5835,1211.7165 188.1871,1211.7165 132.2835,1155.8129 132.2835,1154.2094 189.7906,1211.7165 191.3941,1211.7165 132.2835,1152.6059 132.2835,1151.0024 192.9976,1211.7165 194.6011,1211.7165 132.2835,1149.3989 132.2835,1147.7954 196.2046,1211.7165 197.8082,1211.7165 132.2835,1146.1918 132.2835,1144.5883 199.4117,1211.7165 201.0152,1211.7165 132.2835,1142.9848 132.2835,1141.3813 202.6187,1211.7165 204.2222,1211.7165 132.2835,1139.7778 132.2835,1138.1742 205.8258,1211.7165 207.4293,1211.7165 132.2835,1136.5707 132.2835,1134.9672 209.0328,1211.7165 210.6363,1211.7165 132.2835,1133.3637 132.2835,1131.7602 212.2398,1211.7165 213.8433,1211.7165 132.2835,1130.1567 132.2835,1128.5531 215.4469,1211.7165 217.0504,1211.7165 132.2835,1126.9496 132.2835,1125.3461 218.6539,1211.7165 220.2574,1211.7165 132.2835,1123.7426 132.2835,1122.1391 221.8609,1211.7165 223.4644,1211.7165 132.2835,1120.5356 132.2835,1118.932 225.068,1211.7165 226.6715,1211.7165 132.2835,1117.3285 132.2835,1115.725 228.275,1211.7165 229.8785,1211.7165 132.2835,1114.1215 132.2835,1112.518 231.482,1211.7165 233.0856,1211.7165 132.2835,1110.9144 132.2835,1109.3109 234.6891,1211.7165 236.2926,1211.7165 132.2835,1107.7074 132.2835,1106.1039 237.8961,1211.7165 239.4996,1211.7165 132.2835,1104.5004 132.2835,1102.8969 241.1031,1211.7165 242.7067,1211.7165 132.2835,1101.2933 132.2835,1099.6898 244.3102,1211.7165 245.9137,1211.7165 132.2835,1098.0863 132.2835,1096.4828 247.5172,1211.7165 249.1207,1211.7165 132.2835,1094.8793 132.2835,1093.2758 250.7242,1211.7165 252.3278,1211.7165 132.2835,1091.6722 132.2835,1090.0687 253.9313,1211.7165 255.5348,1211.7165 132.2835,1088.4652 132.2835,1086.8617 257.1383,1211.7165 258.7418,1211.7165 132.2835,1085.2582 132.2835,1083.6546 260.3454,1211.7165 261.9489,1211.7165 132.2835,1082.0511 132.2835,1080.4476 263.5524,1211.7165 265.1559,1211.7165 132.2835,1078.8441 132.2835,1077.2406 266.7594,1211.7165 268.3629,1211.7165 132.2835,1075.6371 132.2835,1074.0335 269.9665,1211.7165 271.57,1211.7165 132.2835,1072.43 132.2835,1070.8265 273.1735,1211.7165 274.777,1211.7165 132.2835,1069.223 132.2835,1067.6195 276.3805,1211.7165 277.984,1211.7165 132.2835,1066.016 132.2835,1064.4124 279.5876,1211.7165 281.1911,1211.7165 132.2835,1062.8089 132.2835,1061.2054 282.7946,1211.7165 284.3981,1211.7165 132.2835,1059.6019 132.2835,1057.9984 286.0016,1211.7165 287.6052,1211.7165 132.2835,1056.3948 132.2835,1054.7913 289.2087,1211.7165 290.8122,1211.7165 132.2835,1053.1878 132.2835,1051.5843 292.4157,1211.7165 294.0192,1211.7165 132.2835,1049.9808 132.2835,1048.3773 295.6227,1211.7165 297.2263,1211.7165 132.283

In [37]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/165_hand_hatch.svg')